In [1]:
import os
import pandas as pd
import re
import psycopg2
import json
from sqlalchemy import create_engine

CSV_DIRECTORY = 'E:/EcoThrift - Preprocessing/raw/'
csv_templates = {}

# CSV Status Dictionary
STATUS_FILE = "csv_files_status.json"
STATUS_FILE_PATH = os.path.join(CSV_DIRECTORY, STATUS_FILE)
csv_files_status = {}

# CSV Templates Dictionary
TEMPLATES_FILE = "csv_templates.json"
TEMPLATES_FILE_PATH = os.path.join(CSV_DIRECTORY, TEMPLATES_FILE)
csv_templates = {}
manifest_cols = [  'order_cde', 'line_number', 'quantity', 'unit_retail_amt', 'description'
                 , 'brand', 'model', 'category', 'subcategory', 'pallet_id', 'upc', 'sku'
                 , 'search_string_1', 'search_string_2', 'search_string_3'
                ]

# Developement
DB_HOST = 'localhost'
DB_NAME = 'EcoThrift - Developement'
DB_USER = 'postgres'
DB_PASSWORD = 'password'

DB_ENGINE = create_engine(f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}')

# Production
'''
    DB_HOST = 'ec2-34-237-167-13.compute-1.amazonaws.com'
    DB_NAME = 'd4op06smk6i192'
    DB_USER = 'u4lpm5bu1q2uan'
    DB_PASSWORD = 'p61f5ea94216c5aba0b43b2cc69c05641f680424e972b7982df23acc4a7a43a16'
'''

# Load CSV Status File
if os.path.exists(STATUS_FILE_PATH):
    with open(STATUS_FILE_PATH, 'r') as file:
        csv_files_status = json.load(file)
    print(f"CSV files status loaded from: {STATUS_FILE_PATH}")
else:    
    print(f"No existing status file found. Starting with an empty dictionary.")

# Update with new files
for file in os.listdir(CSV_DIRECTORY):
    if file.endswith('.csv'):
        if file not in csv_files_status:
            csv_files_status[file] = "Unprocessed"

# Save the updated csv_files_status dictionary to the file
with open(STATUS_FILE_PATH, 'w') as file:
    json.dump(csv_files_status, file)

print(f"CSV files status updated and saved to: {STATUS_FILE_PATH}")

# Load CSV Templates File
if os.path.exists(TEMPLATES_FILE_PATH):
    with open(TEMPLATES_FILE_PATH, 'r') as file:
        csv_templates = json.load(file)
    print(f"CSV templates loaded from: {TEMPLATES_FILE_PATH}")
else:
    print(f"No existing templates file found. Starting with an empty dictionary.")


def process_csv(file, csv_template):
    df = pd.read_csv(os.path.join(CSV_DIRECTORY, file))

    # Apply template transformations to the DataFrame
    for column, transformation in csv_template.items():
        df[column] = eval(transformation)

    # Ensure required columns are present and validated
    required_columns = ['quantity', 'unit_retail_amt', 'description']
    for column in required_columns:
        if column not in df.columns:
            raise ValueError(f"Missing required column: {column}")
        if df[column].isnull().any():
            raise ValueError(f"Empty values found in required column: {column}")

    # Set default values for optional columns
    optional_columns = ['brand', 'model', 'category', 'subcategory', 'pallet_id', 'upc', 'sku', 'search_string_1', 'search_string_2', 'search_string_3']
    for column in optional_columns:
        if column not in df.columns:
            df[column] = ''    

    # Add order_cde column with the CSV file name
    df['order_cde'] = os.path.splitext(file)[0]

    # Add line_number column with the row number
    df['line_number'] = range(1, len(df) + 1)
    
    df = df[manifest_cols]
    display(df.head())

    return df

def CreateCsvTemplate(columns):
    template = {}
    continue_input = True

    while continue_input:
        input_txt = input("Enter a column transformation (col_name=xxxx) or type 'q' to quit: ")

        if input_txt.lower() == 'q':
            continue_input = False
        else:
            try:
                column, transformation = input_txt.split("=", 1)
                column = column.strip()
                transformation = transformation.strip()
                template[column] = transformation
            except ValueError:
                print("Invalid input format. Please enter a valid column transformation.")

    template_key = '| |'.join(template.keys())
    csv_templates[template_key] = template

    print("Template created successfully.")
    print(template)

def load_to_database(df):

    # Load the DataFrame into the manifest table
    df.to_sql('manifest', DB_ENGINE, if_exists='append', index=False)

    print(f"Data loaded to the database successfully.")

CSV files status loaded from: E:/EcoThrift - Preprocessing/raw/csv_files_status.json
CSV files status updated and saved to: E:/EcoThrift - Preprocessing/raw/csv_files_status.json
CSV templates loaded from: E:/EcoThrift - Preprocessing/raw/csv_templates.json


In [2]:
# Process CSV Files
for file in os.listdir(CSV_DIRECTORY):
    if file.endswith('.csv') and csv_files_status[file] == "Unprocessed":
        template_key = '| |'.join(pd.read_csv(os.path.join(CSV_DIRECTORY, file), nrows=0).columns)
        if template_key in csv_templates:
            csv_template = csv_templates[template_key]
            df = process_csv(file, csv_template)
            print("Options:")
            print("1. Continue processing CSV files")
            print("2. Exit")
            choice = input("Enter your choice (1 or 2): ")
            if choice == "2":
                break
            
            # Load df to manifest table
            load_to_database(df)
            
            # Update the status of the processed file            
            csv_files_status[file] = "Processed"
        else:
            print(f"No template found for file: {file}")
            print("Please create a new template using the CreateCsvTemplate function.")
            columns = pd.read_csv(os.path.join(CSV_DIRECTORY, file), nrows=0).columns
            CreateCsvTemplate(columns)
            break

# Save the updated csv_files_status dictionary to the file
with open(STATUS_FILE_PATH, 'w') as file:
    json.dump(csv_files_status, file)
print(f"CSV files status updated and saved to: {STATUS_FILE_PATH}")

# Save the updated csv_templates dictionary to the file
with open(TEMPLATES_FILE_PATH, 'w') as file:
    json.dump(csv_templates, file)
print(f"CSV templates updated and saved to: {TEMPLATES_FILE_PATH}")

,order_cde,line_number,quantity,unit_retail_amt,description,brand,model,category,subcategory,pallet_id,upc,sku,search_string_1,search_string_2,search_string_3
0,CST423585,1,784,23.99,POPMASK HAIR SET P336,POPBAND INC,,Hair Care,HEALTH & BEAUTY AIDS,4922544,,1736860,popmask hair set p336,popband inc popmask hair set p336,hair care health & beauty aids
1,CST423585,2,766,19.99,WET BRUSH,J AND D BRUSH CO LLC,,Hair Care,HEALTH & BEAUTY AIDS,4922544,,1715739,wet brush,j and d brush co llc wet brush,hair care health & beauty aids
2,CST423585,3,144,19.99,CRAYOLA NEON BATH SET,CENTRIC BEAUTY LLC,,Hygiene Products,HEALTH & BEAUTY AIDS,4922544,,1731309,crayola neon bath set,centric beauty llc crayola neon bath set,hygiene products health & beauty aids


Options:
1. Continue processing CSV files
2. Exit
Data loaded to the database successfully.


,order_cde,line_number,quantity,unit_retail_amt,description,brand,model,category,subcategory,pallet_id,upc,sku,search_string_1,search_string_2,search_string_3
0,CST423632,1,207,9.99,DOWDLE PUZZLE ASSORTMENT,AMERICANA ART ENTERPRISES,,Toys,TOYS/SEASONAL,4865135,,888584,dowdle puzzle assortment,americana art enterprises dowdle puzzle assort...,toys toys/seasonal
1,CST423632,2,9,189.99,WASHBURN ACOUSTIC GUITAR,JAM INDUSTRIES USA LLC,,Musical Instruments,MAJOR APPLIANCES,4865135,,1748925,washburn acoustic guitar,jam industries usa llc washburn acoustic guitar,musical instruments major appliances
2,CST423632,3,6,199.99,ROLAND E-X10 KEYBOARD/,ROLAND CORPORATION US,,Musical Instruments,MAJOR APPLIANCES,4865135,,1756655,roland e-x10 keyboard/,roland corporation us roland e-x10 keyboard/,musical instruments major appliances
3,CST423632,4,12,69.99,ASCEND AERONAUTICS,AMAX GROUP USA LLC,,Toys,TOYS/SEASONAL,4865135,,1601380,ascend aeronautics,amax group usa llc ascend aeronautics,toys toys/seasonal
4,CST423632,5,16,49.99,KIDKRAFT BEAT BOARD GAME,KIDKRAFT INC,,Toys,TOYS/SEASONAL,4865135,,1601351,kidkraft beat board game,kidkraft inc kidkraft beat board game,toys toys/seasonal


Options:
1. Continue processing CSV files
2. Exit
CSV files status updated and saved to: E:/EcoThrift - Preprocessing/raw/csv_files_status.json
CSV templates updated and saved to: E:/EcoThrift - Preprocessing/raw/csv_templates.json


In [ ]:
file = os.listdir(CSV_DIRECTORY)[0]

df = pd.read_csv(os.path.join(CSV_DIRECTORY, file))

# Apply template transformations to the DataFrame
for column, transformation in csv_template.items():
    df[column] = eval(transformation)

# Ensure required columns are present and validated
required_columns = ['quantity', 'unit_retail_amt', 'description']
for column in required_columns:
    if column not in df.columns:
        raise ValueError(f"Missing required column: {column}")
    if df[column].isnull().any():
        raise ValueError(f"Empty values found in required column: {column}")

# Set default values for optional columns
optional_columns = ['brand', 'model', 'category', 'subcategory', 'pallet_id', 'upc', 'sku', 'search_string_1', 'search_string_2', 'search_string_3']
for column in optional_columns:
    if column not in df.columns:
        df[column] = ''

# Add order_cde column with the CSV file name
df['order_cde'] = os.path.splitext(file)[0]

# Add line_number column with the row number
df['line_number'] = range(1, len(df) + 1)

In [ ]:
df = df[manifest_cols]
df.head()

In [9]:
from datetime import datetime
from sqlalchemy import create_engine

DB_ENGINE = create_engine('your_database_connection_string')

number = "CST423854"
retail_amt = "7,390"
condition_txt = "A/B"
quantity = "38"
description = "1 Box of Watches & More (TRA-4915347), A/B, 38 Units, Ext. Retail $7,390, Tracy, CA, FIXED PRICE SHIPPING"
price_amt = "$2,025.00"
fee_amt = "$60.75"
shipping_amt = "$35.00"
paid_amt = "$2,120.75"

# Convert data types
retail_amt = float(retail_amt.replace(',', ''))
quantity = int(quantity)
price_amt = float(price_amt.replace('$', '').replace(',', ''))
fee_amt = float(fee_amt.replace('$', '').replace(',', ''))
shipping_amt = float(shipping_amt.replace('$', '').replace(',', ''))
paid_amt = float(paid_amt.replace('$', '').replace(',', ''))

# Generate the INSERT statement
insert_statement = f"""
INSERT INTO public.purchase_order_new (
    "number", retail_amt, condition_txt, quantity, description,
    price_amt, fee_amt, shipping_amt, paid_amt
)
VALUES (
    '{number}', {retail_amt}, '{condition_txt}', {quantity}, '{description}',
    {price_amt}, {fee_amt}, {shipping_amt}, {paid_amt}
);
"""

# Execute the INSERT statement using the DB_ENGINE
with DB_ENGINE.connect() as connection:
    try:
        connection.execute(insert_statement)
        print("Row inserted successfully.")
    except Exception as e:
        print(f"Error inserting row: {str(e)}")

INSERT INTO public.purchase_order_new (
    "number",
    retail_amt,
    condition_txt,
    quantity,
    description,
    price_amt,
    fee_amt,
    shipping_amt,
    paid_amt
)
VALUES (
    'CST423854',
    7390.0,
    'A/B',
    38,
    '1 Box of Watches & More (TRA-4915347), A/B, 38 Units, Ext. Retail $7,390, Tracy, CA, FIXED PRICE SHIPPING',
    2025.0,
    60.75,
    35.0,
    2120.75
);
